In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from factor_data import FactorData
from price_data import PriceData
from factor_model import FactorModel
from pca import PCA
from clustering import Cluster
from find_pairs import PairsTrade
from allocator import Allocator
from pma import PMA


from matplotlib import style
import matplotlib.pyplot as plt
plt.style.use('bmh')

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
%%time

def generate_datelist(start_date, end_date, freq="w"):
    if freq=="w":
        date_list = pd.date_range(start=start_date, end=end_date, 
                             freq='W-MON').strftime('%Y-%m-%d').tolist()
        
    else:
        date_list =[str(d)+"-01" for d in list(pd.period_range(start=start_date, end=end_date, freq='M'))]
    
    return date_list



date_list = generate_datelist("2020-12-01","2022-12-01",freq="m")
pma = PMA(1000000,date_list[0]) # create a pma to record trades

sample_length = 5 # the length of the sample period
display = False # to show the pairs trade graphs



for da in date_list:

    
    print(F"Running ..... {da}")
    
    dt = datetime.strptime(da, '%Y-%m-%d')
    start = str(dt - relativedelta(years=sample_length))[:10] # get start from for the sample period data
    lookback_start = str(dt - relativedelta(months=6))[:10]
    
    # get price data for the sample period
    sample_data = PriceData(["all"], start=start, end=da, ticker_ref_date=da, refresh=False)
    
    # get price data for the out of sample period
    lookback_data = PriceData(["all"], start=lookback_start, end=da, ticker_ref_date=da, refresh=False)
    
    # get the factor data for the sample period
    factors = FactorData("all", frequency="m", start=start, end=da)
    
    # run the ts regression to get factor betas
    _factor_model = FactorModel(
                                sample_data.asset_return, 
                                factors.factor_data,
                                ts_pvalues_threshold=0.1,
                                factor_hard_cap=20,
                                coff_std_limit=5
    )

    # apply PCA to the data
    _pca = PCA(_factor_model.clustering_coefficients)

    # clsuter the data
    cluster = Cluster(
                      _pca.pca,
                      method="GMM",
                      number_of_cluster=None,
                      cluster_cap=1,
                      cluster_floor=1,
                      graph=display
                    )

    # find pairs
    pairs_trade = PairsTrade(
                             _factor_model,
                             cluster, 
                             sample_data, 
                             lookback_data,
                             allocation_date=da,
                             test_percentile=0.5,
                             display=display
                            )


    
    # put the trades into the allocator
    allocator = Allocator(pairs_trade.pairs,trade_date=da)
    
    _info = allocator.info.sample(n=10)
    _orders = allocator.orders.loc[allocator.orders["trade_id"].isin(_info["trade_id"])]
    
    # add the trades into the pma
    pma.add_to_pma(_orders,_info,da)
    
    
    # run pma to calculate aum
    pma.run_pma(da)
    
    
    print("""
    
---------------------------------------------------------------

""")
    

Running ..... 2020-12-01
    563 of ticker.
--------- Runing Cluster 0 ---------------
Sent Trade to Exchange....

        
                  Added 20 Trades, Number of Allocation Increased from 0 to 20
                  Cash Amount went from 1000000 to 1137280.14
                  With Exchange Fee of 4602.94
                  
Date: 2020-12-01 -- AUM: 986807.11

    
---------------------------------------------------------------


Running ..... 2021-01-01
    556 of ticker.
--------- Runing Cluster 0 ---------------
Sent Trade to Exchange....

        
                  Closed 20 Trades, Number of Allocation Reduced from 20 to 0
                  Cash Amount went from 1137280.14 to 1008807.04
                  With Exchange Fee of 4475.68

                  
Sent Trade to Exchange....

        
                  Added 20 Trades, Number of Allocation Increased from 0 to 20
                  Cash Amount went from 1008807.04 to 665692.73
                  With Exchange Fee of 4903.91
 

Date: 2021-12-01 -- AUM: 906439.77

    
---------------------------------------------------------------


Running ..... 2022-01-01
    592 of ticker.
--------- Runing Cluster 0 ---------------
Sent Trade to Exchange....

        
                  Closed 20 Trades, Number of Allocation Reduced from 20 to 0
                  Cash Amount went from 1065039.07 to 900168.61
                  With Exchange Fee of 3674.58

                  
Sent Trade to Exchange....

        
                  Added 20 Trades, Number of Allocation Increased from 0 to 20
                  Cash Amount went from 900168.61 to 260376.64
                  With Exchange Fee of 4313.3
                  
Date: 2022-01-01 -- AUM: 880069.42

    
---------------------------------------------------------------


Running ..... 2022-02-01
    605 of ticker.
--------- Runing Cluster 0 ---------------
Sent Trade to Exchange....

        
                  Closed 20 Trades, Number of Allocation Reduced from 20 to 0
       

Date: 2022-12-01 -- AUM: 769684.84

    
---------------------------------------------------------------


CPU times: user 1h 33min 4s, sys: 39.6 s, total: 1h 33min 43s
Wall time: 1h 33min 32s


In [4]:
pma.aum_history.to_csv("bm-1-historical_aum.csv")
pma.number_of_trade.to_csv("bm-1-number_of_trade.csv")
pma.total_long_position.to_csv("bm-1-long.csv")
pma.total_short_position.to_csv("bm-1-short.csv")